___

In [1]:
# Import libraries to be used

# Warning messages display
# import warnings
# warnings.filterwarnings(action='once') # https://docs.python.org/3/library/warnings.html#the-warnings-filter

# Directories/Files management
import os.path
## from zipfile import ZipFile # De momento no ha hecho falta 

# Timing
import time

# Data analysis and wrangling
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None) # Show all columns in DataFrames
pd.set_option('display.max_rows', 100) # If too high, it greatly slows down the output display and freezes the kernel


# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('ggplot') # choose a style: 'plt.style.available'
sns.set_theme(context='notebook',
              style="darkgrid") # {darkgrid, whitegrid, dark, white, ticks}
palette = sns.color_palette("flare", as_cmap=True);
import altair as alt

# Machine Learning
## from sklearn.[...] import ...

In [2]:
%load_ext memory_profiler

In [3]:
t0 = time.perf_counter() 

In [4]:
# Detect Operating System running and manage paths accordingly

if os.name == 'nt': # Windows
    root = r"C:\Users\turge\CompartidoVM\0.TFM"
    print("Running on Windows.")
elif os.name == 'posix': # Ubuntu
    root = "/home/dsc/shared/0.TFM"
    print("Running on Ubuntu.")
print("root path\t", root)

Running on Windows.
root path	 C:\Users\turge\CompartidoVM\0.TFM


___

## 1. Load the dataset

In [5]:
cols = [
     'MONTH',
     'DAY_OF_MONTH',
     'DAY_OF_WEEK',
     'OP_UNIQUE_CARRIER',
# For the time being, 'TAIL_NUM' will be disregarded due to its high cardinality  (~ 4500) and low expected added value    
#      'TAIL_NUM',
     'ORIGIN',
     'DEST',
#      'CRS_DEP_TIME', # Redundant having the 'DEP_ARR_hour' feature
     'DEP_TIME_hour',
#      'DEP_TIME',
#      'DEP_DELAY',
#      'DEP_DEL15',
#      'TAXI_OUT',
     'TAXI_OUT_median',
#      'TAXI_IN',
     'TAXI_IN_median',
#      'CRS_ARR_TIME', # Redundant having the 'ARR_ARR_hour' feature
     'ARR_TIME_hour', 
#      'ARR_TIME',   
#      'ARR_DELAY',
     'ARR_DEL15', # → Target !!
#      'CANCELLED',
     'CRS_ELAPSED_TIME',
     'DISTANCE',
     'DISTANCE_GROUP',
#      'CARRIER_DELAY',
#      'WEATHER_DELAY',
#      'NAS_DELAY',
#      'SECURITY_DELAY',
#      'LATE_AIRCRAFT_DELAY',
]

In [6]:
preprocessed_input_csv_path = os.path.join(root,
                                           "Output_Data",
                                           "US_DoT",
                                           "AL_OTP_MVP_Preprocessed_19_v2_clean.csv")
preprocessed_input_csv_path

'C:\\Users\\turge\\CompartidoVM\\0.TFM\\Output_Data\\US_DoT\\AL_OTP_MVP_Preprocessed_19_v2_clean.csv'

In [7]:
cols_dtypes = {
               'MONTH' : 'category',
               'DAY_OF_MONTH' : 'category',
               'DAY_OF_WEEK' : 'category',
               'OP_UNIQUE_CARRIER' : 'category',
               'ORIGIN' : 'category',
               'DEST' : 'category',
               'DEP_TIME_hour' : 'category',
               'TAXI_OUT_median' : 'int64',
               'TAXI_IN_median' : 'int64',
               'ARR_TIME_hour' : 'category', 
               'ARR_DEL15' : 'int32', # → Target !!
               'CRS_ELAPSED_TIME' : 'int64',
               'DISTANCE' : 'int64',
               'DISTANCE_GROUP' : 'category'
               }

In [8]:
%%time

df_all = pd.read_csv(preprocessed_input_csv_path,
                     encoding='latin1',
                     usecols=cols,
                     low_memory = False)

Wall time: 18.2 s


In [9]:
# Cast types in accordance to previously defined dictionary:
df_all = df_all.astype(cols_dtypes)
# Display DF columns according to 'cols' order:
df_all = df_all[cols]
df_all

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,ORIGIN,DEST,DEP_TIME_hour,TAXI_OUT_median,TAXI_IN_median,ARR_TIME_hour,ARR_DEL15,CRS_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP
0,1,3,4,9E,TYS,ATL,11,15,8,12,1,70,152,1
1,1,4,5,9E,TYS,ATL,11,15,8,12,1,70,152,1
2,1,5,6,9E,ATL,SGF,9,17,5,10,0,121,563,3
3,1,6,7,9E,ATL,SGF,9,17,5,10,0,123,563,3
4,1,7,1,9E,ATL,SGF,9,17,5,10,0,123,563,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7268227,12,31,2,B6,MCO,SWF,13,15,6,16,1,163,989,4
7268228,12,31,2,B6,DCA,BOS,14,15,6,15,0,90,399,2
7268229,12,31,2,B6,PHL,BOS,7,17,6,8,0,85,280,2
7268230,12,31,2,B6,BOS,SJU,8,16,5,13,0,242,1674,7


In [10]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7268232 entries, 0 to 7268231
Data columns (total 14 columns):
 #   Column             Dtype   
---  ------             -----   
 0   MONTH              category
 1   DAY_OF_MONTH       category
 2   DAY_OF_WEEK        category
 3   OP_UNIQUE_CARRIER  category
 4   ORIGIN             category
 5   DEST               category
 6   DEP_TIME_hour      category
 7   TAXI_OUT_median    int64   
 8   TAXI_IN_median     int64   
 9   ARR_TIME_hour      category
 10  ARR_DEL15          int32   
 11  CRS_ELAPSED_TIME   int64   
 12  DISTANCE           int64   
 13  DISTANCE_GROUP     category
dtypes: category(9), int32(1), int64(4)
memory usage: 325.8 MB


In [11]:
# from pandas_profiling import ProfileReport

# # # Complete report:
# # prof = ProfileReport(df_all)
# # prof.to_file('informe.html')

# # Sample report (more computationally efficient)
# prof = ProfileReport(df_all.sample(10000)) 
# prof.to_file('sample_report.html')

In [12]:
# from pandas_profiling import ProfileReport

# # Complete report:
# prof = ProfileReport(df_all, minimal=True)
# prof.to_file('complete_report.html')

# # # Sample report (more computationally efficient)
# # prof = ProfileReport(df_all.sample(10000)) 
# # prof.to_file('sample_report.html')

## 2. Dataset overview

In [13]:
# For the initial runs, define a reasonably-sized sample:
sample_length = int(1e6)

# The dataset is somehow imbalanced (80% on-time / 20% delayed), so let's perform some quick undersampling:
delayed = df_all[df_all['ARR_DEL15'] == 1].sample(sample_length // 2)
ontime = df_all[df_all['ARR_DEL15'] == 0].sample(sample_length // 2)
df = delayed.append(ontime)

# Get dummies for the categorical features:
df = pd.get_dummies(df)

print("On-time flights: {:7d} ({:5.2f}%)".format(len(ontime), 100*len(ontime)/len(df)))
print("Delayed flights: {:7d} ({:5.2f}%)".format(len(delayed), 100*len(delayed)/len(df)))

On-time flights:  500000 (50.00%)
Delayed flights:  500000 (50.00%)


### Train / Test / Validation split
Break the dataset into three blocks:
1. Training (70%)
2. Test (15%)
3. Validation (15%)

In [14]:
# Divide the data:
train = df.sample(frac=0.7)
test_valid = df.drop(train.index)
test = test_valid.sample(frac=0.5)
valid = test_valid.drop(test.index)

print("Total dataset:      {:7d} ({:6.2f}%)".format(len(df), 100*len(df)/len(df)))
print("Training dataset:   {:7d} ( {:5.2f}%)".format(len(train), 100*len(train)/len(df)))
print("Test dataset:       {:7d} ( {:5.2f}%)".format(len(test), 100*len(test)/len(df)))
print("Validation dataset: {:7d} ( {:5.2f}%)".format(len(valid), 100*len(valid)/len(df)))

Total dataset:      1000000 (100.00%)
Training dataset:    700000 ( 70.00%)
Test dataset:        150000 ( 15.00%)
Validation dataset:  150000 ( 15.00%)


# XGBoost

In [15]:
from xgboost import XGBClassifier

In [16]:
# Train / Test partition:
X_train, y_train = train.drop("ARR_DEL15", axis=1), train["ARR_DEL15"]
X_test, y_test = test.drop("ARR_DEL15", axis=1), test["ARR_DEL15"]

# Validation:
X_valid, y_valid = valid.drop("ARR_DEL15", axis=1), valid["ARR_DEL15"]

In [17]:
# Basic model definition:
xgb_model = XGBClassifier(use_label_encoder=False, verbosity=1, random_state=0)

## GridSearchCV

In [18]:
# Parameter grid:
parameters = {
            "n_estimators":[100, 200, 300, 400, 500],
            "max_depth": [4, 5, 6],
            "learning_rate":[0.0001, 0.001, 0.01, 0.1],
            "objective": ["binary:logistic"],
            "booster":["gbtree"],
            "tree_method":['auto']
             }

In [19]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV

# clf = GridSearchCV(xgb_model, parameters, n_jobs=5, 
#                    cv=StratifiedKFold(n_splits=5, shuffle=True), 
#                    scoring='f1', refit=True)

clf = RandomizedSearchCV(xgb_model, parameters, n_jobs=5, 
                         cv=StratifiedKFold(n_splits=5, shuffle=True), 
                         scoring='f1', refit=True, n_iter=10)

clf.fit(X_train, y_train, eval_metric='logloss')

print("Scorer:", clf.scorer_)
print("Best score:", clf.best_score_)
print("Best parameters:", clf.best_params_)
print("Best estimator:", clf.best_estimator_)
# print("CV results:", clf.cv_results_)

MemoryError: Unable to allocate 446. MiB for an array with shape (836, 560000) and data type uint8

## Evaluation

In [ ]:
# Evaluate the model's metrics:

from sklearn.metrics import fbeta_score, f1_score, recall_score, precision_score, accuracy_score, \
                            confusion_matrix, roc_curve, roc_auc_score, plot_roc_curve

def clf_metrics(classifier, y_test, y_pred):
    print(type(classifier.best_estimator_))
    print(confusion_matrix(y_test, y_pred))
    print("F-beta (ß=2) = {:6.3f}".format(fbeta_score(y_test, y_pred, beta=2)))   
    print("F1 =           {:6.3f}".format(f1_score(y_test, y_pred)))   
    print("Recall =       {:6.3f}".format(recall_score(y_test, y_pred)))   
    print("Precision =    {:6.3f}".format(precision_score(y_test, y_pred)))   
    print("Accuracy =     {:6.3f}".format(accuracy_score(y_test, y_pred)))

### Test

In [ ]:
probabilities = clf.predict_proba(X_test)[:,1]
probabilities

In [ ]:
y_pred = [1 if i > .5 else 0 for i in probabilities] # '.5' representa el threshold
y_pred[:10]

In [ ]:
clf_metrics(clf, y_test, y_pred)

In [ ]:
fpr, tpr, _ = roc_curve(y_test,  probabilities)
auc = roc_auc_score(y_test, probabilities)

In [ ]:
# Plot the figure:
fig, ax = plt.subplots(figsize=(6,6))

plot_roc_curve(clf, X_test, y_test, color='blue', ax=ax, name='XGBoost_test')
plt.plot([0,1], [0,1], color='orange', linestyle='--')

plt.xticks(np.arange(0.0, 1.1, step=0.1))
plt.xlabel("fpr", fontsize=15)

plt.yticks(np.arange(0.0, 1.1, step=0.1))
plt.ylabel("tpr", fontsize=15)

plt.title('ROC Curve Analysis', fontweight='bold', fontsize=15)
plt.legend(prop={'size':13}, loc='lower right')

### Validation

In [ ]:
probabilities_val = clf.predict_proba(X_valid)[:,1]
probabilities_val

In [ ]:
y_val_pred = [1 if i > .5 else 0 for i in probabilities_val] # '.5' representa el threshold
y_val_pred[:10]

In [ ]:
clf_metrics(clf, y_valid, y_val_pred)

In [ ]:
fpr, tpr, _ = roc_curve(y_valid,  probabilities_val)
auc = roc_auc_score(y_valid, probabilities_val)

In [ ]:
# Plot the figure:
fig, ax = plt.subplots(figsize=(6,6))

plot_roc_curve(clf, X_valid, y_valid, color='red', ax=ax, name='XGBoost_valid')
plt.plot([0,1], [0,1], color='orange', linestyle='--')

plt.xticks(np.arange(0.0, 1.1, step=0.1))
plt.xlabel("fpr", fontsize=15)

plt.yticks(np.arange(0.0, 1.1, step=0.1))
plt.ylabel("tpr", fontsize=15)

plt.title('ROC Curve Analysis', fontweight='bold', fontsize=15)
plt.legend(prop={'size':13}, loc='lower right')

### Comparison

In [ ]:
print("TEST dataset results")
clf_metrics(clf, y_test, y_pred)
print("-"*50)
print("VALIDATION dataset results")
clf_metrics(clf, y_valid, y_val_pred)

In [ ]:
# Plot the figure:
fig, ax = plt.subplots(figsize=(6,6))

plot_roc_curve(clf, X_valid, y_valid, color='red', ax=ax, name='XGBoost_valid')
plot_roc_curve(clf, X_test, y_test, color='blue', ax=ax, name='XGBoost_test')
plt.plot([0,1], [0,1], color='orange', linestyle='--')

plt.xticks(np.arange(0.0, 1.1, step=0.1))
plt.xlabel("fpr", fontsize=15)

plt.yticks(np.arange(0.0, 1.1, step=0.1))
plt.ylabel("tpr", fontsize=15)

plt.title('ROC Curve Analysis', fontweight='bold', fontsize=15)
plt.legend(prop={'size':13}, loc='lower right')

___

In [ ]:
t1 = time.perf_counter() - t0
print("Time elapsed: {:2.0f}h {:2.0f}min {:2.0f}s".format(t1//3600, (t1%3600)//60, (t1%3600)%60))

___